In [28]:
import os
import pyspark
conf = pyspark.SparkConf()

conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)

spark = pyspark.SQLContext.getOrCreate(sc)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_125/1107090999.py:8 

In [29]:
from pyspark.sql.functions import countDistinct,desc,filter,col,split,monotonically_increasing_id
from pyspark.sql import SparkSession


In [30]:
def line22csv(line,tags_list):
    offset=0
    result=""
    for i in tags_list:
        val=""
        patt=i+"="
        ind=line.find(patt,offset)
        if(ind==-1):
            result+=','
            continue
        ind+=(len(i)+2)
        val+='\"'
        while(line[ind]!='\"'):
            val+=line[ind]
            ind+=1
        val+='\"'
        result+=val+','
        offset=ind
    return result[:-1]

In [31]:
tags_list=['Id','PostTypeId','AcceptedAnswerId','ParentId','CreationDate',
           'DeletionDte','Score','ViewCount','Body','OwnerUserId','OwnerDisplayName',
           'LastEditorUserId','LastEditorDisplayName','LastEditDate','LastActivityDate',
           'Title','Tags','AnswerCount','CommentCount','FavoriteCount','ClosedDate',
           'CommunityOwnedDate']

In [32]:
fileName = '*se_Posts.xml'

In [33]:
raw = (sc.textFile(fileName, 4))

In [34]:
headers = raw.take(2)
PostsRDD = raw.filter(lambda x: x != headers)

In [35]:
Posts_csvRDD=PostsRDD.map(lambda x:line22csv(x,tags_list))

In [38]:
targetFile = './cs&se_posts_csv'

In [39]:
Posts_csvRDD.saveAsTextFile(targetFile)